In [47]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [48]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWM1MzU2M2ItMWNlMS00ZjJhLTg3NDMtMzVhZmI1OTNjMTY0IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI2SGpUcDZvTHBDUnNOUkZDUjY5ZUFWQ1hvYXdrNlJ3SlBIQzNNdmJoeVZudSJdLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [49]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZGQ2ODRiZTktNGZlZi00MzhjLWE2ZGItYWJhMDE4YmMzZGIxIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI5UjIzQ0dvZGN3UkFUQjV2aEJweFBGdkw5dWl6SERTZ3QzM1pUdlFVbjk3YSJdLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==


In [50]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': 'dd684be9-4fef-438c-a6db-aba018bc3db1',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['9R23CGodcwRATB5vhBpxPFvL9uizHDSgt33ZTvQUn97a'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [51]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 3HcsNeByCMCsvqVXm5Zc3mMa69WxjrqXRrmccERjhiCcvbXBe5sDhEDNZXQ8DXuo5v1TREmzwwTprYqCUaHQZY2b

Public key: 
 ByJjrACzCUqjWQBjbAZaiopAJE1A4gTdP8MH8iNWHL5d


**Sending a connection request to acapy**

In [52]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [53]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [54]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

### FEATURE DISCOVERY

In [58]:
pp.pprint(util.getFeatureDiscovery(agents[0]))

{'@id': '67ec276b-cc7c-4508-8b72-e5b473e788a6',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/present-proof/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-payments/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0'},
               {'pid': 'https://didcomm.org/action-menu/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/action-menu/1.0',
                'roles': ['provider']},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/introduction-service/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/

### Notes

* To receive invitation use "new connections" protocol "receive invitation"
* To create invitation use create invitation from "invitations" protocol

### Done
* create invitation
* receive invitation
* basic message
    * get
    * send
    * basic message
    * deletes
* dids
    * list dids
    

### Todo
* basic message sorting
* add problem report for did on ledger
* add problem report for schema send

In [56]:
# message = util.buildMessage(
#         'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/create-invitation',
#         label="ConnectionBetweenAgents",
#         auto_accept='auto',
#         alias="agent1ToAgent2"
#     )
# createInvitation = util.sendMessage(message, agents[0])

In [57]:
# createInvitation

In [64]:
message = util.buildMessage(
        'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/schema-exchange/1.0/send',
        payload="asdasdad"
    )
schemaExchange = util.sendMessage(message, agents[0])

KeyboardInterrupt: 

In [63]:
schemaExchange

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/schema-exchange/1.0/send',
 '@id': '6bb18882-393b-403e-a9d1-c240d3931287',
 '~thread': {'thid': 'c476d97f-bee0-4de5-9bc4-e2fab43a6000'},
 'hashid': 'ec103814b9ccb1e305ce20d6a25e681d3735d2301d5bb631b8dfa0abb2394a99',
 'payload': 'asdasdad'}

In [23]:
pp.pprint(util.connectAgents(agents[0], agents[1]))

{'@id': '3ef00dd2-27dc-43ab-bbc9-cc575b7c8e14',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/connection',
 'connection_id': '7f583c69-3b04-4c3a-a9a8-54f1636db137',
 'label': 'ConnectionBetweenAgents',
 'my_did': 'V992yuD4YTv1s6SSfLLnkn',
 'raw_repr': {'accept': 'auto',
              'connection_id': '7f583c69-3b04-4c3a-a9a8-54f1636db137',
              'created_at': '2020-06-09 14:56:40.330661Z',
              'initiator': 'external',
              'invitation_key': '6EbsPk8Z91YKgAb2KHtdi1jP6uHJTU4mpFTchkkbXH6k',
              'invitation_mode': 'once',
              'my_did': 'V992yuD4YTv1s6SSfLLnkn',
              'request_id': '725e0e06-59af-4d34-836f-d875b4609a9d',
              'routing_state': 'none',
              'state': 'request',
              'their_label': 'ConnectionBetweenAgents',
              'updated_at': '2020-06-09 14:56:40.359881Z'},
 'state': 'pending'}


In [24]:
pp.pprint(util.getConnections(agents[1]))

{'@id': '5966f953-d429-4a5f-a0a1-8258a3665cc3',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/list',
 'connections': [{'connection_id': '7f583c69-3b04-4c3a-a9a8-54f1636db137',
                  'label': 'ConnectionBetweenAgents',
                  'my_did': 'V992yuD4YTv1s6SSfLLnkn',
                  'raw_repr': {'accept': 'auto',
                               'connection_id': '7f583c69-3b04-4c3a-a9a8-54f1636db137',
                               'created_at': '2020-06-09 14:56:40.330661Z',
                               'initiator': 'external',
                               'invitation_key': '6EbsPk8Z91YKgAb2KHtdi1jP6uHJTU4mpFTchkkbXH6k',
                               'invitation_mode': 'once',
                               'my_did': 'V992yuD4YTv1s6SSfLLnkn',
                               'request_id': '725e0e06-59af-4d34-836f-d875b4609a9d',
                               'routing_state': 'none',
                               'sta

In [25]:
newDid = util.createDid(agents[0])
pp.pprint(newDid['result'])

{'did': 'UTckeR8yEogf7QUDKNoFH7',
 'verkey': 'Fy2kCRmHd76UGHw8Z3bpKofzwMn9WmRXs4P9pbUFBtTR'}


In [26]:
util.registerDidOnLedger(agents[0], newDid['result']['did'], newDid['result']['verkey'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response',
 '@id': '93a27c5e-10d6-4978-9771-e416d67aa157',
 '~thread': {'thid': '725e0e06-59af-4d34-836f-d875b4609a9d'},
 'connection~sig': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single',
  'signature': 'kN1rtDUQFA6Hk7WrF-vl-fpuvk5TXfpPQewpgDGEkIfVVU7IJJE2sMgJa_FgBTs2nbXU-SBlNocwQwjkA3LHDg==',
  'sig_data': 'AAAAAF7foyh7IkRJRCI6ICJKZWZlZXhXRlN0S01QRVp4UUtMU3FpIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpKZWZlZXhXRlN0S01QRVp4UUtMU3FpIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6SmVmZWV4V0ZTdEtNUEVaeFFLTFNxaSMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OkplZmVleFdGU3RLTVBFWnhRS0xTcWkiLCAicHVibGljS2V5QmFzZTU4IjogIkFjdzdWSnpzaHk2ajdEMXZqWjg3cXg0UkVCYjFNRndUa0tXOU55ZVVDV3c1In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMTgiLCAicHVibGljS2V5IjogImRpZDpzb3Y6SmVmZWV4V0ZTdEt

In [27]:
util.setPublicDid(agents[0], newDid['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '7afbcb60-0807-4d4b-a7e0-785a61391bc3',
 '~thread': {'thid': '237b349d-ac0c-4f8e-a0c1-8497fadd82d6'},
 'result': {'metadata': {'public': True},
  'did': 'UTckeR8yEogf7QUDKNoFH7',
  'verkey': 'Fy2kCRmHd76UGHw8Z3bpKofzwMn9WmRXs4P9pbUFBtTR'}}

In [28]:
pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

{'@id': '6ea282cb-58b9-43df-a6e2-9b874f21a66a',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schemas/0.1/schema-id',
 'schema_id': "('UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0', {'ver': '1.0', "
              "'id': 'UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0', 'name': "
              "'schesssdddvd', 'version': '1.0', 'attrNames': "
              "['memesssaaaaaaa'], 'seqNo': 11})",
 '~thread': {'thid': '3c40a5f4-3784-4f9d-a5a3-0ff9e5a602e8'}}


In [29]:
result = util.schemaGetList(agents[0])
last_schema_id = result['results'][-1]['schema_id']
print(result)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schemas/0.1/schema-list', '@id': 'b70eec03-206f-4ca2-a2a7-8d6f0eef2829', '~thread': {'thid': 'fffe2a09-4741-455a-a921-a7253f37f3bd'}, 'results': [{'state': 'written', 'attributes': ['memesssaaaaaaa'], 'schema_name': 'schesssdddvd', 'created_at': '2020-06-09 14:56:43.771284Z', 'schema_version': '1.0', 'schema_id': 'UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0', 'updated_at': '2020-06-09 14:56:43.771284Z', 'author': 'self'}]}


In [30]:
util.schemaGet(agents[0], last_schema_id)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schemas/0.1/schema',
 '@id': '6032790e-0e04-41db-b0d5-08c806209b25',
 '~thread': {'thid': '016ce3e5-a806-4dbc-b0c2-74d781ffd3fa'},
 'state': 'written',
 'attributes': ['memesssaaaaaaa'],
 'schema_name': 'schesssdddvd',
 'created_at': '2020-06-09 14:56:43.854957Z',
 'schema_version': '1.0',
 'updated_at': '2020-06-09 14:56:43.854957Z',
 'schema_id': 'UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0',
 'author': 'other'}

In [31]:
result = util.sendCredentialDefinition(agents[0], last_schema_id)
cred_def_id = result['cred_def_id']
print(cred_def_id)

UTckeR8yEogf7QUDKNoFH7:3:CL:11:schesssdddvd_1.0


In [32]:
util.getCredentialDefinitionList(agents[0])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-credential-definitions/0.1/credential-definition-list',
 '@id': '4a3aaacc-4069-472b-ac7b-583f8966aa74',
 '~thread': {'thid': '316991ab-c20d-4050-8b9e-4fd89623976d'},
 'results': [{'cred_def_id': 'UTckeR8yEogf7QUDKNoFH7:3:CL:11:schesssdddvd_1.0',
   'state': 'written',
   'attributes': ['memesssaaaaaaa'],
   'created_at': '2020-06-09 14:56:46.810226Z',
   'schema_id': 'UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0',
   'updated_at': '2020-06-09 14:56:46.810226Z',
   'author': 'self'}]}

In [33]:
util.getCredentialDefinition(agents[0], cred_def_id)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-credential-definitions/0.1/credential-definition',
 '@id': 'e347f6f3-3b05-46f6-aff9-a0aca47d9221',
 '~thread': {'thid': '81d8203c-3a79-4c5a-a459-4f668c15eed7'},
 'cred_def_id': 'UTckeR8yEogf7QUDKNoFH7:3:CL:11:schesssdddvd_1.0',
 'state': 'written',
 'attributes': ['memesssaaaaaaa'],
 'created_at': '2020-06-09 14:56:46.905245Z',
 'updated_at': '2020-06-09 14:56:46.905245Z',
 'schema_id': 'UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0',
 'author': 'other'}